<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Executing udf on VantageCloud Lake</b>
</header>

### Disclaimer
The sample code (“Sample Code”) provided is not covered by any Teradata agreements. Please be aware that Teradata has no control over the model responses to such sample code and such response may vary. The use of the model by Teradata is strictly for demonstration purposes and does not constitute any form of certification or endorsement. The sample code is provided “AS IS” and any express or implied warranties, including the implied warranties of merchantability and fitness for a particular purpose, are disclaimed. In no event shall Teradata be liable for any direct, indirect, incidental, special, exemplary, or consequential damages (including, but not limited to, procurement of substitute goods or services; loss of use, data, or profits; or business interruption) sustained by you or a third party, however caused and on any theory of liability, whether in contract, strict liability, or tort arising in any way out of the use of this sample code, even if advised of the possibility of such damage.

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Introduction</b><br>
* udf() function creates a user-defined Python function which runs on each teradataml DataFrame row.
* The function returns a teradataml DataFrame Column, and can be passed to teradataml DataFrame.assign(). 
* UDF leverages Apply table operator for running the function VantageCloud Lake.

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Import the required libraries</b>

In [2]:
# Importing required libraries
import getpass
import random
from teradataml import *
from teradataml.dataframe.functions import udf
from teradatasqlalchemy.types import INTEGER, DATE

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Connect to Vantage</b>

In [3]:
# Create the connection.
td_context = create_context(host=getpass.getpass("Hostname: "), username=getpass.getpass("Username: "), password=getpass.getpass("Password: "))

Hostname:  ········
Username:  ········
Password:  ········


In [4]:
# Set the Authentication Token
set_config_params(auth_token=getpass.getpass("auth_data: "), ues_url=getpass.getpass("ues_url: "))

auth_data:  ········
ues_url:  ········


True

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Loading Dataset</b>

In [5]:
# Load the example dataset.
load_example_data("dataframe", ["sales"])
df = DataFrame('sales')

# Preparing the data 
df_sales = df.assign(id = case([(df.accounts == 'Alpha Co', random.randrange(1, 9)),
                                   (df.accounts == 'Blue Inc', random.randrange(1, 9)),
                                   (df.accounts == 'Jones LLC', random.randrange(1, 9)),
                                   (df.accounts == 'Orange Inc', random.randrange(1, 9)),
                                   (df.accounts == 'Yellow Inc', random.randrange(1, 9)),
                                   (df.accounts == 'Red Inc', random.randrange(1, 9))]))

In [6]:
df_sales

accounts,Feb,Jan,Mar,Apr,datetime,id
Jones LLC,200.0,150,140,180,04/01/2017,5
Orange Inc,210.0,None,None,250,04/01/2017,4
Alpha Co,210.0,200,215,250,04/01/2017,2
Blue Inc,90.0,50,95,101,04/01/2017,1
Yellow Inc,90.0,None,None,None,04/01/2017,4
Red Inc,200.0,150,140,None,04/01/2017,3


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Setting up the Environment</b>

In [7]:
list_user_envs()

,env_name,env_description,base_env_name,language,conda
0,conda_env_3_10_demo,Conda environment for notebook demo,python_3.10,python,True
1,demo_env,Demo env 1.,python_3.10,Python,False
2,non_conda_env_3_8_demo,Non Conda environment for notebook demo,python_3.8,Python,False
3,openml_env,DONT DELETE: OpenML environment,python_3.10,Python,False
4,openml_env_dhan,DONT DELETE: OpenML environment,python_3.10,Python,False
5,testenv,This env 'testenv' is created with base env 'p...,python_3.10,Python,False


In [8]:
# create a new environment use create_env function, if environment is already created use get_env to get access to the environment.
env = create_env('testudf', 'python_3.10', 'Test environment for UDF')

User environment 'testudf' created.


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>UDF to upper case values in column 'accounts'.</b>

udf to get the values in 'accounts' to upper case without passing any arguments.
If no ``env_name`` is provided, udf use the default `openml_env` user environment.

In [9]:
@udf
def to_upper(s):
        return s.upper()

In [10]:
#Assign the Column Expression returned by user defined function to the DataFrame.
out_df = df_sales.assign(upper_col = to_upper('accounts'))

# Print the result.
out_df

accounts,Feb,Jan,Mar,Apr,datetime,id,upper_col
Orange Inc,210.0,None,None,250,17/01/04,4,ORANGE INC
Alpha Co,210.0,200,215,250,17/01/04,2,ALPHA CO
Yellow Inc,90.0,None,None,None,17/01/04,4,YELLOW INC
Red Inc,200.0,150,140,None,17/01/04,3,RED INC
Jones LLC,200.0,150,140,180,17/01/04,5,JONES LLC
Blue Inc,90.0,50,95,101,17/01/04,1,BLUE INC


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>UDF to get the factorial of values in column 'id'.</b>

udf to get the factorial of values in 'id'and store the result in INTEGER type column.

In [11]:
@udf(returns = INTEGER(), env_name = 'testudf')
def factorial(n):
    import math
    return math.factorial(n)

In [12]:
#Assign the Column Expression returned by user defined function to the DataFrame.
out_df = df_sales.assign(fact = factorial('id'))

# Print the result.
out_df

accounts,Feb,Jan,Mar,Apr,datetime,id,fact
Orange Inc,210.0,None,None,250,17/01/04,4,24
Jones LLC,200.0,150,140,180,17/01/04,5,120
Alpha Co,210.0,200,215,250,17/01/04,2,2
Yellow Inc,90.0,None,None,None,17/01/04,4,24
Blue Inc,90.0,50,95,101,17/01/04,1,1
Red Inc,200.0,150,140,None,17/01/04,3,6


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>UDF to add 30 days to values in column 'datetime'.</b>

udf to add 30 to the 'datetime' column and store the result in DATE type column.

In [13]:
@udf(returns = DATE(), env_name = 'testudf')
def updated_date(x, y):
    import datetime
    return (datetime.datetime.strptime(x, "%Y-%m-%d")+datetime.timedelta(y)).strftime("%Y-%m-%d")

In [14]:
#Assign the Column Expression returned by user defined function to the DataFrame.
out_df = df_sales.assign(new_date = updated_date('datetime', 30))

# Print the result.
out_df

accounts,Feb,Jan,Mar,Apr,datetime,id,new_date
Jones LLC,200.0,150,140,180,17/01/04,5,17/02/03
Blue Inc,90.0,50,95,101,17/01/04,1,17/02/03
Red Inc,200.0,150,140,None,17/01/04,3,17/02/03
Yellow Inc,90.0,None,None,None,17/01/04,4,17/02/03
Alpha Co,210.0,200,215,250,17/01/04,2,17/02/03
Orange Inc,210.0,None,None,250,17/01/04,4,17/02/03


<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Disconnect from Vantage</b>

In [15]:
remove_context()

True